# Задание 2

In [36]:
%%capture
!pip install compress-fasttext[full]


## Построить и оценить качество бейзлайна

In [37]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import compress_fasttext

# Models from sklearn
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Model Evaluations
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import RocCurveDisplay as plot_roc_curve

In [6]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/data_vacancies.csv'

Mounted at /content/drive


In [58]:
df = pd.read_csv(file_path)

In [39]:
small_model = compress_fasttext.models.CompressedFastTextKeyedVectors.load(
    'https://github.com/avidale/compress-fasttext/releases/download/gensim-4-draft/geowac_tokens_sg_300_5_2020-100K-20K-100.bin'
)

## Измерение качества константного предсказания

Когда говорят о "константном предсказании" в контексте регрессии, это обычно означает использование постоянного значения в качестве предсказания для всех наблюдений. Обычно это может быть среднее значение или медиана зависимой переменной.

Произведение измерения качества константного предсказания включает в себя оценку того, насколько хорошо это постоянное предсказание соответствует фактическим значениям. We will use:
Mean Absolute Error (MAE):

In [8]:
from IPython.display import display, Math, Latex

formula = '\\frac{\sum_{i=1}^n |y_i - \\hat{y}_i|}{n}'

latex_obj = Math(formula)

display(latex_obj)

<IPython.core.display.Math object>

In [9]:
df.columns.tolist()

['id',
 'custom_position',
 'schedule',
 'salary_from',
 'salary_to',
 'salary_pay_type',
 'offer_education_id',
 'education_name',
 'education_is_base',
 'education_order_num',
 'city_id',
 'list_regions',
 'work_skills',
 'tags_id']

## Кодирование категориальных столбцов

**Выбор столбцов для использования**

```
['id',
 'custom_position',
 'schedule',
 'salary_from',
 'salary_to',
 'salary_pay_type',
 'offer_education_id',
 'education_name',
 'education_is_base',
 'education_order_num',
 'city_id',
 'list_regions',
 'work_skills',
 'tags_id',
 'target']
```

The dataset we are using now is from one company: Rabota.ru. But because for the future we will collect data from three different sources. We need to select columns that are present in all of the sources.

We will then keep:

- custom_position
- schedule
- salary_from
- salary_to
- city_id

Нашей target будет salary_from.

Существует два основных способа кодировать категориальные признаки, такие как 'schedule' и 'city_id' для задач машинного обучения:

One-hot кодирование: One-hot кодирование создает новый двоичный признак для каждой уникальной категории в категориальном признаке. Например, если признак 'schedule' имеет три категории (полный рабочий день, неполный рабочий день, договор), one-hot кодирование создаст три новых двоичных признака: schedule_full_time, schedule_part_time и schedule_contract. Каждый признак будет 0 или 1, в зависимости от того, присутствует ли соответствующая категория в наблюдении.

One-hot кодирование является распространенным и эффективным способом кодирования категориальных признаков, но оно может быть расточительным для памяти и вычислительных ресурсов, особенно для признаков с множеством категорий.

Label кодирование: Label кодирование назначает числовое значение каждой уникальной категории в категориальном признаке. Это числовое значение обычно является индексом категории, но оно также может быть настраиваемым сопоставлением. Например, если признак 'schedule' имеет три категории (полный рабочий день, неполный рабочий день, договор), label кодирование может сопоставлять full-time с 0, part-time с 1 и contract с 2.

Для признаков 'schedule' и 'city_id' в наборе данных вакансий, вероятно, хорошим выбором будет one-hot кодирование. Оба признака имеют относительно небольшое количество категорий, и one-hot кодирование позволит модели изучить взаимосвязи между категориями и заработной платой.

### **Custom Position**

Чтобы закодировать предложения на кириллице в столбце "custom_position" для использования в алгоритме прогнозирования зарплаты, вы можете использовать техники текстового кодирования. Один из распространенных методов - использование векторных представлений слов, таких как FasstText, или более продвинутых языковых моделей.

Векторные представления слов, такие как FastText, позволяют алгоритму понимать семантические отношения между словами, представляя их в непрерывном векторном пространстве. Закодированные векторы могут быть использованы в качестве признаков в модели машинного обучения для прогнозирования зарплаты. Другие методы, такие как использование предварительно обученных языковых моделей (например, BERT, GPT), могут быть также рассмотрены в зависимости от сложности и конкретных требований вашей задачи.

Затем мы создадим столбцов для каждого числа в списке.

In [40]:
def get_vector(value):
  """Returns the vector representation of the given value."""
  vector = small_model.get_vector(value)
  return vector

In [59]:
df_encode = df.copy()
selected_columns = ['schedule', 'salary_to', 'salary_from', 'city_id', 'custom_position']
df_encode = df_encode[selected_columns]

In [60]:
df_encode['custom_position'] = df_encode['custom_position'].apply(get_vector)

In [61]:
df_encode

,schedule,salary_to,salary_from,city_id,custom_position
0,полный рабочий день,120000,60000,2,"[-0.16283481, 0.21359403, 0.15451327, -0.12198..."
1,полный рабочий день,120000,60000,2,"[-0.12818882, 0.12597714, 0.28937253, -0.00347..."
2,полный рабочий день,80000,60000,2,"[-0.20935018, 0.104254976, 0.19449428, -0.0129..."
3,частичная занятость,35000,30000,1,"[-0.36603537, 0.031681407, 0.28992158, -0.1233..."
4,частичная занятость,35000,30000,57,"[-0.36603537, 0.031681407, 0.28992158, -0.1233..."
...,...,...,...,...,...
19484,полный рабочий день,70000,45000,1,"[-0.42711735, 0.43605497, 0.40649128, -0.31359..."
19485,сменный график,58000,35000,1,"[-0.28695458, -0.10356837, 0.20561127, 0.11603..."
19486,полный рабочий день,77000,77000,1,"[-0.2390533, -0.03135372, 0.22209024, 0.087064..."
19487,полный рабочий день,120000,80000,2,"[-0.4367608, 0.20505118, 0.22864334, 0.1721858..."


In [62]:
df_encode['custom_position'] = df_encode['custom_position'].values.ravel()

In [63]:
%time
df_ = pd.DataFrame(df_encode['custom_position'].to_list(), columns=[f'vector_{i}' for i in range(len(df_encode['custom_position'].iloc[0]))])
df_1 = pd.concat([df_encode, df_], axis=1).drop("custom_position", axis=1)

CPU times: user 8 µs, sys: 1 µs, total: 9 µs
Wall time: 16.2 µs


In [64]:
df_1

,schedule,salary_to,salary_from,city_id,vector_0,vector_1,vector_2,vector_3,vector_4,vector_5,...,vector_290,vector_291,vector_292,vector_293,vector_294,vector_295,vector_296,vector_297,vector_298,vector_299
0,полный рабочий день,120000,60000,2,-0.162835,0.213594,0.154513,-0.121982,-0.029939,-0.304156,...,-0.291821,-0.172719,0.007797,0.004109,-0.300479,0.245348,0.217966,-0.198783,-0.128744,0.043903
1,полный рабочий день,120000,60000,2,-0.128189,0.125977,0.289373,-0.003476,-0.129682,-0.224158,...,-0.299037,-0.221662,-0.061972,-0.206315,-0.108113,0.203288,0.166086,-0.032407,-0.065472,0.117336
2,полный рабочий день,80000,60000,2,-0.209350,0.104255,0.194494,-0.012942,-0.053993,-0.200087,...,-0.147017,-0.142724,-0.103702,-0.010637,-0.158289,0.249718,0.134893,-0.207344,-0.005284,0.059902
3,частичная занятость,35000,30000,1,-0.366035,0.031681,0.289922,-0.123331,-0.076429,-0.400840,...,-0.202899,0.090761,0.142858,-0.067872,-0.275784,0.097124,0.299659,-0.385510,-0.000062,0.101015
4,частичная занятость,35000,30000,57,-0.366035,0.031681,0.289922,-0.123331,-0.076429,-0.400840,...,-0.202899,0.090761,0.142858,-0.067872,-0.275784,0.097124,0.299659,-0.385510,-0.000062,0.101015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19484,полный рабочий день,70000,45000,1,-0.427117,0.436055,0.406491,-0.313595,-0.176224,-0.249900,...,-0.371014,0.015965,0.026756,-0.159011,-0.146671,0.134393,0.629227,-0.012031,-0.086118,-0.069761
19485,сменный график,58000,35000,1,-0.286955,-0.103568,0.205611,0.116038,-0.081405,0.058864,...,-0.115339,-0.047851,0.259571,0.205588,-0.009814,0.024699,0.141673,-0.054753,-0.236569,-0.096308
19486,полный рабочий день,77000,77000,1,-0.239053,-0.031354,0.222090,0.087064,-0.143939,-0.063031,...,0.057307,0.060856,0.003304,-0.073683,0.007051,-0.020302,0.147459,0.023199,-0.107903,0.145099
19487,полный рабочий день,120000,80000,2,-0.436761,0.205051,0.228643,0.172186,-0.124017,-0.293177,...,-0.196022,0.191928,-0.229226,-0.038024,-0.037084,-0.123508,0.290207,-0.167918,0.048910,0.162911


### Encode city_id and schedule

In [65]:
def preprocess_data(df):
    df_encoded = pd.get_dummies(df, columns=['city_id', 'schedule'], prefix=['city', 'schedule'])

    df_encoded['target'] = df_encoded['salary_from']
    df_encoded.drop('salary_from', axis=1, inplace=True)

    return df_encoded

df = preprocess_data(df_1)

In [67]:
df

,salary_to,vector_0,vector_1,vector_2,vector_3,vector_4,vector_5,vector_6,vector_7,vector_8,...,city_269,city_270,city_272,schedule_вахта,schedule_полный рабочий день,schedule_свободный график,schedule_сменный график,schedule_удаленная работа,schedule_частичная занятость,target
0,120000,-0.162835,0.213594,0.154513,-0.121982,-0.029939,-0.304156,0.059934,0.031059,0.211994,...,0,0,0,0,1,0,0,0,0,60000
1,120000,-0.128189,0.125977,0.289373,-0.003476,-0.129682,-0.224158,0.146302,0.046938,0.219321,...,0,0,0,0,1,0,0,0,0,60000
2,80000,-0.209350,0.104255,0.194494,-0.012942,-0.053993,-0.200087,0.004927,0.204725,0.301456,...,0,0,0,0,1,0,0,0,0,60000
3,35000,-0.366035,0.031681,0.289922,-0.123331,-0.076429,-0.400840,0.065848,0.136194,0.324135,...,0,0,0,0,0,0,0,0,1,30000
4,35000,-0.366035,0.031681,0.289922,-0.123331,-0.076429,-0.400840,0.065848,0.136194,0.324135,...,0,0,0,0,0,0,0,0,1,30000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19484,70000,-0.427117,0.436055,0.406491,-0.313595,-0.176224,-0.249900,0.206578,-0.142696,0.408901,...,0,0,0,0,1,0,0,0,0,45000
19485,58000,-0.286955,-0.103568,0.205611,0.116038,-0.081405,0.058864,-0.049900,0.112408,-0.121614,...,0,0,0,0,0,0,1,0,0,35000
19486,77000,-0.239053,-0.031354,0.222090,0.087064,-0.143939,-0.063031,-0.026878,0.143475,0.222062,...,0,0,0,0,1,0,0,0,0,77000
19487,120000,-0.436761,0.205051,0.228643,0.172186,-0.124017,-0.293177,0.028297,0.123710,0.592853,...,0,0,0,0,1,0,0,0,0,80000


## Произведено разбиение датасета на тернировочную/тестовую выборки - 2 балла

In [69]:
X = df.drop(['target', 'salary_to'], axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2024)

## Произведено измерение качества на отложенной выборке с использованием ранее выбранной метрики

In [70]:
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=2024)

rf_regressor.fit(X_train, y_train)

y_pred = rf_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')

Mean Absolute Error (MAE): 11737.866047260168


In [73]:
np.random.seed(2024)